# Using Snowpark to implement ZSTD dictionary decompression
Snowflake has compress and decompress function that support ZSTD, but it does not support a user-supplied dictionary file. This code demonstrates:
1. Connecting to Snowflake using the Snowpark for Python connector.
2. Creating fake data using the Faker library and creating a ZSTD dictionary file. 
3. Uploading that faked & compressed to a Snowflake table.
4. Upload our dictionary file to a stage and create a Python UDF to decompress data using that file.
5. Calling that Python UDF using ordinary SQL


## 1. Connect to Snowflake using the Snowpark for Python connector

In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import pandas_udf
from snowflake.snowpark.types import PandasSeries
import streamlit as st
import pandas as pd

from faker import Faker
import pyzstd

from cachetools import cached
import sys
import os

import json 
with open(".env/creds.json") as f:
   connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()

# test if we have a connection
session.sql("select current_warehouse() wh, current_database() db, current_schema() schema, current_version() v").show()


2023-02-27 12:18:53.270 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.15, Platform: macOS-10.16-x86_64-i386-64bit
2023-02-27 12:18:53.271 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-02-27 12:18:53.271 INFO    snowflake.connector.connection: Setting use_openssl_only mode to False
2023-02-27 12:18:54.228 INFO    snowflake.snowpark.session: Snowpark Session information: 
"version" : 1.1.0,
"python.version" : 3.8.15,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 647878641528874,
"os.name" : Darwin

2023-02-27 12:18:54.234 INFO    snowflake.connector.cursor: query: [select current_warehouse() wh, current_database() db, current_schema() schema, c...]
2023-02-27 12:18:54.37

-------------------------------------------------------
|"WH"    |"DB"                     |"SCHEMA"  |"V"    |
-------------------------------------------------------
|XSMALL  |COMMODITIES_FORECASTING  |PUBLIC    |7.6.3  |
-------------------------------------------------------



## 2. Create synthetic data and create a ZSTD dictionary.

### Create 10,000 fake address records, encoded as bytes. 

In [2]:
fake = Faker()
fake_addresses = [bytes(fake.address(), 'utf-8') for i in range(10000)]
fake_addresses

[b'894 Paul Burg Suite 710\nSouth Loriport, IA 92529',
 b'6968 Valdez Mall Suite 439\nKatherinechester, WA 16098',
 b'226 Harmon Bridge\nLake Jeffreymouth, NE 92689',
 b'04553 Samantha Isle\nChurchberg, OK 57659',
 b'196 Martin Extensions\nLeetown, AK 91519',
 b'175 Gilbert Vista\nJonesborough, OR 81410',
 b'PSC 9582, Box 5880\nAPO AP 35855',
 b'785 Porter Square\nRachelmouth, AK 04106',
 b'Unit 8425 Box 6088\nDPO AE 38956',
 b'120 Natalie Tunnel\nNew Jason, AL 95365',
 b'9311 Carpenter Cove\nEast Whitney, LA 57352',
 b'295 Turner Groves\nStanleymouth, ND 07992',
 b'204 Cesar Parks\nSimonport, NE 18070',
 b'602 Caitlin Roads Suite 036\nLake Jessica, TN 25089',
 b'PSC 9867, Box 9093\nAPO AP 84443',
 b'55316 Costa Mission Apt. 385\nWest Robert, PA 95399',
 b'91147 Kathleen Wells Suite 149\nNew Jillstad, AZ 04768',
 b'959 Young Neck Suite 393\nHayesville, DC 32915',
 b'353 Eaton Summit\nDiazberg, MO 52127',
 b'1644 Wheeler Way Suite 310\nSouth Henrytown, AZ 30007',
 b'824 Smith Lane\nAmym

### Create ZSTD dictionary file, compress fake addresses, and write compressed addresses to a Snowflake table.

In [3]:
# create dictionary
zstd_dict = pyzstd.train_dict(fake_addresses, 100*1024)

# write our dictionary to a file
with open("zstd_address_dictionary.d", "wb") as f:
    f.write(zstd_dict.dict_content)

In [4]:
compressed_fake_addresses = [pyzstd.compress(i, zstd_dict=zstd_dict) for i in fake_addresses]
compressed_fake_addresses



[b'(\xb5/\xfd#i\xe6\xebY0\xf5\x00\x00\xd3\x80\x02\x8f\x8b_\x18\x8d\x88\xa4A.\x01\x04\xfc\xc9R\x9aEM \xf2\xfe\x80{\x1eNXU\x01',
 b'(\xb5/\xfd#i\xe6\xebY5\xed\x00\x00\x03\x81\x02\xfe\xc6W\x18\x15\xe9S\x9d\x9f\x03\x04\xfc\x9d8\xf8\xbc\x9d\x17\x9e\xd1KG\xe3e\x1a\x01',
 b'(\xb5/\xfd#i\xe6\xebY-\xcd\x00\x00\xb3\x00\x02\xef\x8c_!\x1c\x91\x8fQ\x04\xfc\x8f(\x02&c\xcd^WP\x80N\x1e',
 b'(\xb5/\xfd#i\xe6\xebY(\xed\x00\x003\x81\x03\xbf\xdc[#%<VF.<2\xb9\xab\x16\x03\xfc+\x19\xf9\xabX\x07\xeb\x10\xb5,',
 b"(\xb5/\xfd#i\xe6\xebY'\xb5\x00\x00s\x00\x01\x7f{\xff\x17\x04\xfclh\xbd\n\xe24Z\x1e\xc6\x83\x99\x9a\xaf",
 b"(\xb5/\xfd#i\xe6\xebY(\xbd\x00\x00\xb3\xc0\x01vz^'%\xdb\x17\x03\xfcdL\xe8\x06K7h8U\x8d\x08",
 b'(\xb5/\xfd#i\xe6\xebY\x1f\xad\x00\x00\xc3\xc0\x01\xbb\xbeY&\xbe\xe8\x1b\x03\xfc\x93\xe2\xa6X\xc8I8iE',
 b"(\xb5/\xfd#i\xe6\xebY'\xc5\x00\x00\x83@\x01l\xa7\xd2z\x07\x04\xfc:\x80\xc1\xd9H@N\xe1\x00\x0f\x89\xb4\xce\x01",
 b'(\xb5/\xfd#i\xe6\xebY\x1f\xb5\x00\x00\xd3\x00\x02\xbc\xefY!\xc6\x8b\xea\x01\x03\

In [5]:
# write our fake addresses to a Snowflake table
sp_df = session.create_dataframe(compressed_fake_addresses, schema=["comressed_address"])
sp_df.write.mode('overwrite').save_as_table("compressed_addresses")

2023-02-27 12:18:56.006 INFO    snowflake.connector.cursor: query: [SELECT NULL :: BINARY AS "COMRESSED_ADDRESS"]
2023-02-27 12:18:56.057 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:18:56.058 INFO    snowflake.connector.cursor: query: [CREATE  OR  REPLACE  SCOPED TEMPORARY  TABLE SNOWPARK_TEMP_TABLE_MLMYGPL37T("COM...]
2023-02-27 12:18:56.402 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:18:56.461 INFO    snowflake.connector.cursor: query: [INSERT  INTO SNOWPARK_TEMP_TABLE_MLMYGPL37T("COMRESSED_ADDRESS") VALUES (?)]
2023-02-27 12:18:58.041 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:18:58.042 INFO    snowflake.connector.cursor: query: [CREATE  OR  REPLACE    TABLE  compressed_addresses AS  SELECT  *  FROM ( SELECT ...]
2023-02-27 12:18:59.123 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:18:59.124 INFO    snowflake.connector.cursor: query: [DROP  TABLE  If  EXISTS SNOWPARK_TEMP_TABLE

## Deploy decompression UDF for our data using the dictionary

### Upload dictionary and create UDF

In [6]:
# create a stage to hold our dictionary file and UDF
session.sql("""create stage if not exists zstd_decompress""").collect()

2023-02-27 12:18:59.311 INFO    snowflake.connector.cursor: query: [create stage if not exists zstd_decompress]
2023-02-27 12:18:59.453 INFO    snowflake.connector.cursor: query execution done


[Row(status='ZSTD_DECOMPRESS already exists, statement succeeded.')]

In [7]:
# write our dictionary file to our stage
# this is our dictionary for our Faker data. you'll want to use your own dictionary
session.file.put("zstd_address_dictionary.d", "@zstd_decompress", overwrite=True)

2023-02-27 12:18:59.474 INFO    snowflake.connector.cursor: query: [PUT 'file://zstd_address_dictionary.d' '@zstd_decompress'  parallel = 4 source_c...]
2023-02-27 12:18:59.562 INFO    snowflake.connector.cursor: query execution done


[PutResult(source='zstd_address_dictionary.d', target='zstd_address_dictionary.d.gz', source_size=102400, target_size=44928, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

### Create a UDF
Our UDF loads our dictionary file from the stage and uses the pyzstd library to decompress a binary field and return a string. 

We use a pandas_udf with inputs and outputs of a Pandas series. This is a [Vectorized UDF](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-udfs#label-snowpark-python-udf-vectorized) which enables better performance. 

In [8]:
@cached(cache={})
def read_file(filename):
       import_dir = sys._xoptions.get("snowflake_import_directory")
       if import_dir:
              with open(os.path.join(import_dir, filename), 'rb') as file:
                     m = file.read()
                     return m

@pandas_udf(name="address_decompress", 
     is_permanent=True,
     stage_location="@zstd_decompress",
     imports=["@zstd_decompress/zstd_address_dictionary.d"], 
     packages=["pyzstd", "cachetools"],
     replace=True
     )
def address_decompress(input_df:PandasSeries[bytes])-> PandasSeries[str]:
    d = read_file("zstd_address_dictionary.d") 
    zstd_dict = pyzstd.ZstdDict(d)
    output_series = input_df.apply(lambda x: pyzstd.decompress(x, zstd_dict=zstd_dict).decode('utf-8'))
    return output_series 

2023-02-27 12:19:00.289 INFO    snowflake.connector.cursor: query: [ls '@zstd_decompress']
2023-02-27 12:19:00.513 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:19:00.515 INFO    snowflake.connector.cursor: query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01aa9dab-0000-fdc8-00...]
2023-02-27 12:19:01.136 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:19:01.142 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM information_schema.packages]
2023-02-27 12:19:01.238 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:19:01.241 INFO    snowflake.connector.cursor: query: [SELECT "PACKAGE_NAME", array_agg("VERSION") AS "ARRAY_AGG(VERSION)" FROM ( SELEC...]
2023-02-27 12:19:02.021 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:19:02.022 WARNING snowflake.snowpark.session: The version of package pyzstd in the local environment is 0.15.4, which does not fit the criteria for the 

## Call our Python UDF using normal SQL

In [9]:
session.sql("select COMRESSED_ADDRESS, address_decompress(COMRESSED_ADDRESS) from compressed_addresses").show()

2023-02-27 12:19:14.412 INFO    snowflake.connector.cursor: query: [select COMRESSED_ADDRESS, address_decompress(COMRESSED_ADDRESS) from compressed_...]
2023-02-27 12:19:15.278 INFO    snowflake.connector.cursor: query execution done
2023-02-27 12:19:15.284 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM (select COMRESSED_ADDRESS, address_decompress(COMRESSED_ADDRESS)...]
2023-02-27 12:19:16.746 INFO    snowflake.connector.cursor: query execution done


------------------------------------------------------------------------------------------------
|"COMRESSED_ADDRESS"                                 |"ADDRESS_DECOMPRESS(COMRESSED_ADDRESS)"  |
------------------------------------------------------------------------------------------------
|bytearray(b'(\xb5/\xfd#i\xe6\xebY0\xf5\x00\x00\...  |894 Paul Burg Suite 710                  |
|                                                    |South Loriport, IA 92529                 |
|bytearray(b'(\xb5/\xfd#i\xe6\xebY*\xcd\x00\x00\...  |5366 Stacey Views                        |
|                                                    |North Jennifer, NM 51254                 |
|bytearray(b"(\xb5/\xfd#i\xe6\xebY$\xd5\x00\x00\...  |011 Richard Isle                         |
|                                                    |Brownbury, WI 36991                      |
|bytearray(b'(\xb5/\xfd#i\xe6\xebY-\xcd\x00\x00\...  |65752 Samantha Divide                    |
|                             